In [ ]:
import numpy as np
import cv2
from pygame import mixer
import time
mixer.init()
sound = mixer.Sound('beep-01a.wav')

In [ ]:
with_glasses=np.load('with_glasses.npy')
without_glasses=np.load('without_glasses.npy')

In [ ]:
with_glasses.shape

In [ ]:
without_glasses.shape

In [ ]:
with_glasses=with_glasses.reshape(200,50*50*3)

In [ ]:
without_glasses=without_glasses.reshape(201,50*50*3)

In [ ]:
with_glasses.shape

In [ ]:
without_glasses.shape

In [ ]:
X=np.r_[with_glasses,without_glasses]

In [ ]:
X.shape

In [ ]:
from numpy import zeros
y=np.zeros(X.shape[0])

In [ ]:
y[200:]=1.0

In [ ]:
names={0:'WITH GLASSES',1:'WITHOUT GLASSES'}

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

In [ ]:
pca = PCA(n_components=3)
x_train = pca.fit_transform(x_train)

In [ ]:
x_train[0]

In [ ]:
x_train.shape

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.15)

In [ ]:
svm=SVC()
svm.fit(x_train,y_train)

In [ ]:
#x_test = pca.transform(x_test)
y_pred = svm.predict(x_test)
accuracy_score(y_test,y_pred)

In [ ]:
haar_data=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
capture=cv2.VideoCapture(0)
data=[]
font=cv2.FONT_HERSHEY_COMPLEX
while True:
    flag,img=capture.read()
    
    if flag:
        faces=haar_data.detectMultiScale(img)
        
        for x,y,w,h in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,215,255),5)
            face = img[y:y+h,x:x+w,:]
            face = cv2.resize(face,(50,50))
            face= face.reshape(1,-1)
            pred=svm.predict(face)
            
            n=names[int(pred)]
            
            cv2.putText(img,n,(x,y),font,1,(244,250,250),3)
            if(n=='WITHOUT GLASSES'):
                try:
                    sound.play()
                except: 
                    isplaying = False
                    pass
            print(n)
            
            
        cv2.imshow('result',img)
            
        if cv2.waitKey(2)==27 or len(data) >=200:
            break
            
cv2.destroyAllWindows()
capture.release()